In [ ]:
%pip install dotenv

In [2]:
# Atur Timezone
# %pip install pytz

# Untuk Melakukan Geocoding
%pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [6]:
# Standard library imports
import pandas as pd
import numpy as np
import pandas_gbq as gbq
import requests

# Datetime and timezone library
from datetime import datetime, timezone
from pytz import timezone as tz

# Geocoding library
from geopy.geocoders import Nominatim

# Google API library
from google.oauth2 import service_account

# Logging
import logging
# Setup logger
logging.basicConfig(level=logging.INFO)

Table of Content
1. Extract
2. Transform
3. Load

# **1. Data Extraction**

In [4]:
# Define BASE_URL

# BASE URL for real-time earthquake data (past 24 hours)
BASE_URL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson"

# BASE URL for historical earthquake data
BASE_URL_HISTORICAL = "https://earthquake.usgs.gov/fdsnws/event/1/query"

In [17]:
# Get all real-time earthquake data
response = requests.get(BASE_URL)
data = response.json()

In [28]:
data['features'][0]["properties"]

{'mag': 1.23,
 'place': '18 km NE of Coachella, CA',
 'time': 1752031823770,
 'updated': 1752032032187,
 'tz': None,
 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ci41017575',
 'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/ci41017575.geojson',
 'felt': None,
 'cdi': None,
 'mmi': None,
 'alert': None,
 'status': 'automatic',
 'tsunami': 0,
 'sig': 23,
 'net': 'ci',
 'code': '41017575',
 'ids': ',ci41017575,',
 'sources': ',ci,',
 'types': ',nearby-cities,origin,phase-data,scitech-link,',
 'nst': 48,
 'dmin': 0.1306,
 'rms': 0.15,
 'gap': 40,
 'magType': 'ml',
 'type': 'earthquake',
 'title': 'M 1.2 - 18 km NE of Coachella, CA'}

In [31]:
# Convert JSON to DataFrame

records = []

for feature in data['features']:
    props = feature['properties']
    coords = feature['geometry']['coordinates']  # [lon, lat, depth]

    record = {
        "id": feature.get("id"),
        "place": props.get("place"),
        "mag": props.get("mag"),
        "time": pd.to_datetime(props.get("time"), unit='ms'),
        "updated": pd.to_datetime(props.get("updated"), unit='ms'),
        "tz": props.get("tz"),
        "felt": props.get("felt"),
        "cdi": props.get("cdi"),
        "mmi": props.get("mmi"),
        "alert": props.get("alert"),
        "status": props.get("status"),
        "tsunami": props.get("tsunami"),
        "sig": props.get("sig"),
        "net": props.get("net"),
        "code": props.get("code"),
        "ids": props.get("ids"),
        "sources": props.get("sources"),
        "types": props.get("types"),
        "longitude": coords[0],
        "latitude": coords[1],
        "depth": coords[2],
        "fetched_at": datetime.now(timezone.utc)
    }
    records.append(record)

records_df = pd.DataFrame(records)

In [32]:
records_df

,id,place,mag,time,updated,tz,felt,cdi,mmi,alert,...,sig,net,code,ids,sources,types,longitude,latitude,depth,fetched_at
0,ci41017575,"18 km NE of Coachella, CA",1.23,2025-07-09 03:30:23.770,2025-07-09 03:33:52.187,None,NaN,NaN,NaN,None,...,23,ci,41017575,",ci41017575,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",-116.029000,33.782000,1.040,2025-07-09 03:42:03.014802+00:00
1,nc75207327,"2 km E of Pinnacles, CA",1.87,2025-07-09 03:27:19.310,2025-07-09 03:30:28.520,None,NaN,NaN,NaN,None,...,54,nc,75207327,",nc75207327,",",nc,",",focal-mechanism,nearby-cities,origin,phase-data,",-121.122833,36.535999,9.200,2025-07-09 03:42:03.015177+00:00
2,nc75207322,"8 km NNW of The Geysers, CA",0.68,2025-07-09 03:22:44.550,2025-07-09 03:24:19.610,None,NaN,NaN,NaN,None,...,7,nc,75207322,",nc75207322,",",nc,",",nearby-cities,origin,phase-data,",-122.815002,38.828499,1.410,2025-07-09 03:42:03.015562+00:00
3,uw62136042,"22 km ENE of Ashford, Washington",0.36,2025-07-09 03:17:23.080,2025-07-09 03:18:56.910,None,NaN,NaN,NaN,None,...,2,uw,62136042,",uw62136042,",",uw,",",origin,phase-data,",-121.768166,46.846832,-1.720,2025-07-09 03:42:03.016002+00:00
4,nc75207317,"4 km NNW of The Geysers, CA",0.75,2025-07-09 03:17:09.340,2025-07-09 03:18:44.596,None,NaN,NaN,NaN,None,...,9,nc,75207317,",nc75207317,",",nc,",",nearby-cities,origin,phase-data,",-122.778999,38.805168,2.370,2025-07-09 03:42:03.016515+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
447,nc75206902,"4 km NNW of The Geysers, CA",1.03,2025-07-08 04:11:39.870,2025-07-08 04:13:13.724,None,NaN,NaN,NaN,None,...,16,nc,75206902,",nc75206902,",",nc,",",nearby-cities,origin,phase-data,",-122.783997,38.803833,3.180,2025-07-09 03:42:03.091575+00:00
448,av93661711,"7 km W of Akutan, Alaska",0.64,2025-07-08 04:10:43.050,2025-07-08 21:22:40.670,None,NaN,NaN,NaN,None,...,6,av,93661711,",av93661711,",",av,",",origin,phase-data,",-165.888333,54.143000,4.810,2025-07-09 03:42:03.091685+00:00
449,us7000qba0,"29 km NNW of Diphu, India",4.20,2025-07-08 03:52:10.603,2025-07-08 04:08:33.040,None,NaN,NaN,NaN,None,...,271,us,7000qba0,",us7000qba0,",",us,",",origin,phase-data,",93.361700,26.102100,10.000,2025-07-09 03:42:03.091785+00:00
450,nc75206892,"6 km WSW of Gilroy, CA",1.47,2025-07-08 03:52:05.750,2025-07-08 04:12:20.273,None,NaN,NaN,NaN,None,...,33,nc,75206892,",nc75206892,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",-121.622833,36.972668,3.700,2025-07-09 03:42:03.091896+00:00


# **2. Transformation**

- Basic Cleaning
- Data Enrichment

In [33]:
df = records_df.copy()

In [34]:
# Drop duplicate records
df = df.drop_duplicates(subset=["id"])

# Drop rows with null magnitude or location
df = df.dropna(subset=["mag", "place", "latitude", "longitude", "time"])

In [35]:
# Convert time columns to datetime if not already
# Convert UTC time columns to datetime with Asia/Jakarta timezone
WIB = tz('Asia/Jakarta')
df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['updated'] = pd.to_datetime(df['updated'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['fetched_at'] = pd.to_datetime(df['fetched_at'], errors='coerce').dt.tz_convert('UTC').dt.tz_convert(WIB)

In [36]:
#  Filter only valid magnitudes
df = df[df['mag'] >= 0]

# Select relevant columns
selected_columns = [
    "id", "place", "mag", "time", "updated",
    "latitude", "longitude", "depth",
    "tsunami", "sig", "status", "alert", "types",
    "felt", "cdi", "mmi", "fetched_at"
]
df = df[selected_columns]

In [37]:
# 2b. Data Enrichment

# Add Magnitude Category
def classify_magnitude(mag):
        if mag < 2.0:
            return "Micro"
        elif mag < 4.0:
            return "Minor"
        elif mag < 5.5:
            return "Light"
        elif mag < 7.0:
            return "Moderate"
        elif mag < 8.0:
            return "Strong"
        else:
            return "Major"

df['mag_category'] = df['mag'].apply(classify_magnitude)

In [38]:
df.head()

,id,place,mag,time,updated,latitude,longitude,depth,tsunami,sig,status,alert,types,felt,cdi,mmi,fetched_at,mag_category
0,ci41017575,"18 km NE of Coachella, CA",1.23,2025-07-09 10:30:23.770000+07:00,2025-07-09 10:33:52.187000+07:00,33.782000,-116.029000,1.04,0,23,automatic,None,",nearby-cities,origin,phase-data,scitech-link,",NaN,NaN,NaN,2025-07-09 10:42:03.014802+07:00,Micro
1,nc75207327,"2 km E of Pinnacles, CA",1.87,2025-07-09 10:27:19.310000+07:00,2025-07-09 10:30:28.520000+07:00,36.535999,-121.122833,9.20,0,54,automatic,None,",focal-mechanism,nearby-cities,origin,phase-data,",NaN,NaN,NaN,2025-07-09 10:42:03.015177+07:00,Micro
2,nc75207322,"8 km NNW of The Geysers, CA",0.68,2025-07-09 10:22:44.550000+07:00,2025-07-09 10:24:19.610000+07:00,38.828499,-122.815002,1.41,0,7,automatic,None,",nearby-cities,origin,phase-data,",NaN,NaN,NaN,2025-07-09 10:42:03.015562+07:00,Micro
3,uw62136042,"22 km ENE of Ashford, Washington",0.36,2025-07-09 10:17:23.080000+07:00,2025-07-09 10:18:56.910000+07:00,46.846832,-121.768166,-1.72,0,2,automatic,None,",origin,phase-data,",NaN,NaN,NaN,2025-07-09 10:42:03.016002+07:00,Micro
4,nc75207317,"4 km NNW of The Geysers, CA",0.75,2025-07-09 10:17:09.340000+07:00,2025-07-09 10:18:44.596000+07:00,38.805168,-122.778999,2.37,0,9,automatic,None,",nearby-cities,origin,phase-data,",NaN,NaN,NaN,2025-07-09 10:42:03.016515+07:00,Micro


In [39]:
df["place"]

0             18 km NE of Coachella, CA
1               2 km E of Pinnacles, CA
2           8 km NNW of The Geysers, CA
3      22 km ENE of Ashford, Washington
4           4 km NNW of The Geysers, CA
                     ...               
447         4 km NNW of The Geysers, CA
448            7 km W of Akutan, Alaska
449           29 km NNW of Diphu, India
450              6 km WSW of Gilroy, CA
451     37 km ESE of Malaga, New Mexico
Name: place, Length: 422, dtype: object

In [40]:
# Add address details
def get_address_detail(latitude, longitude):
    result = {}
    coder = Nominatim(user_agent="myGeocoder", timeout=3)
    # print(coder.reverse(f"{latitude}, {longitude}"))
    address = coder.reverse(f"{latitude}, {longitude}")

    result["city"] = address.raw.get("address").get("city") if address else np.nan
    result["state"] = address.raw.get("address").get("state") if address else np.nan
    result["country"] = address.raw.get("address").get("country") if address else np.nan
    return result

address_detail = df.apply(lambda row: get_address_detail(row['latitude'], row['longitude']), axis=1, result_type='expand')
df = pd.concat([df, address_detail], axis=1)

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 422 entries, 0 to 451
Data columns (total 21 columns):
 #   Column        Non-Null Count  Dtype                       
---  ------        --------------  -----                       
 0   id            422 non-null    object                      
 1   place         422 non-null    object                      
 2   mag           422 non-null    float64                     
 3   time          422 non-null    datetime64[ns, Asia/Jakarta]
 4   updated       422 non-null    datetime64[ns, Asia/Jakarta]
 5   latitude      422 non-null    float64                     
 6   longitude     422 non-null    float64                     
 7   depth         422 non-null    float64                     
 8   tsunami       422 non-null    int64                       
 9   sig           422 non-null    int64                       
 10  status        422 non-null    object                      
 11  alert         2 non-null      object                      
 12 

In [42]:
# Add time-based columns
df['day_of_week'] = df['time'].dt.day_name()
df['hour_of_day'] = df['time'].dt.hour

In [43]:
# 2c. Save data into CSV
df.to_csv(f"../data/earthquakes-{datetime.now().strftime('%Y%m%d')}.csv", index=False)

In [44]:
# BigQuery configurations
PROJECT_ID = "earthquake-jcds0612"
TABLE_ID = "earthquakes.events" # dataset.id
CREDENTIALS_PATH = "../config/earthquake-jcds0612-28c50709cff0.json"

In [45]:
credentials = service_account.Credentials.from_service_account_file(CREDENTIALS_PATH)

In [46]:
gbq.to_gbq(
    dataframe=df,
    destination_table=TABLE_ID,
    project_id=PROJECT_ID,
    credentials=credentials,
    if_exists="replace"
)

422 out of 422 rows loaded.<?, ?it/s]INFO:pandas_gbq.gbq_connector:
100%|██████████| 1/1 [00:00<00:00, 1416.04it/s]
